In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
import os
import pandas as pd
#os.environ['SPARK_HOME']="C:/Users/321ni/spark-2.4.3-bin-hadoop2.7/"
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
#os.environ['PYSPARK_DRIVER_PYTHON_OPS']='lab'
os.environ['PYSPARK_PYTHON']='python'

In [2]:
import findspark
#findspark.init("C:/Users/321ni/spark-2.4.3-bin-hadoop2.7/")
findspark.find()

'C:\\Users\\321ni\\spark-3.5.0-bin-hadoop3'

In [3]:
spark = SparkSession.builder.master("local[*]").config("spark.driver.memory", "4g").appName("sparkan").getOrCreate()

In [6]:
mvcdata2 = spark.read.format('csv').options(header='true',inferschema='true').load("nonull.csv")
#mvcdata2.show()

In [7]:
mvcdata2.createOrReplaceTempView("allData")

In [8]:
mvcdata2.count()

1839003

In [9]:
yearly_accidents_sql = "SELECT YEAR(CAST(TO_DATE(CRASH_DATE,'MM/dd/yyyy') as DATE)),COUNT(*) FROM allData GROUP BY YEAR(CAST(TO_DATE(CRASH_DATE,'MM/dd/yyyy') as DATE))"

In [10]:
yearly_accidents_df = spark.sql(yearly_accidents_sql)

In [11]:
yearly_accidents_df.show()

+---------------------------------------------------+--------+
|year(CAST(to_date(CRASH_DATE, MM/dd/yyyy) AS DATE))|count(1)|
+---------------------------------------------------+--------+
|                                               2018|  218580|
|                                               2023|   78205|
|                                               2022|   96526|
|                                               2019|  197752|
|                                               2020|  105892|
|                                               2012|   85382|
|                                               2017|  218404|
|                                               2021|  103537|
|                                               2015|  182772|
|                                               2014|  172545|
|                                               2016|  207678|
|                                               2013|  171730|
+---------------------------------------------------+--

In [12]:
timely_accidents_sql = "SELECT HOUR(CAST(CRASH_TIME as TIMESTAMP)) as TIME_OCCUR,COUNT(*) as TOTAL_COUNT FROM allData GROUP BY HOUR(CAST(CRASH_TIME as TIMESTAMP)) ORDER BY TOTAL_COUNT DESC"

In [13]:
timely_accidents_df = spark.sql(timely_accidents_sql)

In [14]:
timely_accidents_df.show()

+----------+-----------+
|TIME_OCCUR|TOTAL_COUNT|
+----------+-----------+
|        16|     132323|
|        17|     129358|
|        14|     122880|
|        15|     115021|
|        18|     113619|
|        13|     106502|
|        12|     101502|
|         8|     101097|
|         9|      97602|
|        11|      95489|
|        19|      93527|
|        10|      91399|
|        20|      78292|
|        21|      66432|
|        22|      60428|
|         0|      59473|
|         7|      55564|
|        23|      50610|
|         6|      40308|
|         1|      31639|
+----------+-----------+
only showing top 20 rows



In [15]:

month_year_time_sql = "SELECT HOUR(CAST(CRASH_TIME as TIMESTAMP)) as TIME_OCCUR, "
month_year_time_sql += " YEAR(CAST(TO_DATE(CRASH_DATE,'MM/dd/yyyy') as DATE)) YEAR,"
month_year_time_sql += " MONTH(CAST(TO_DATE(CRASH_DATE,'MM/dd/yyyy') as DATE)) MONTH,"
month_year_time_sql += "  COUNT(*) from allData GROUP BY TIME_OCCUR,MONTH,YEAR"

In [16]:
month_year_time_df = spark.sql(month_year_time_sql)
month_year_time_df.show()

+----------+----+-----+--------+
|TIME_OCCUR|YEAR|MONTH|count(1)|
+----------+----+-----+--------+
|        11|2021|    7|     443|
|         2|2021|    7|     209|
|         0|2022|    2|     332|
|         5|2021|   10|     215|
|         8|2021|    2|     309|
|         1|2020|    2|     225|
|         1|2020|    3|     142|
|         9|2019|   12|     822|
|        10|2022|    4|     337|
|         6|2022|    1|     250|
|        16|2022|    9|     538|
|        13|2019|   12|     907|
|         8|2021|    7|     376|
|        18|2020|    8|     575|
|        11|2020|    4|     155|
|         8|2020|    3|     530|
|        14|2019|    5|    1249|
|         8|2022|    2|     382|
|        15|2021|    8|     567|
|        12|2021|    5|     493|
+----------+----+-----+--------+
only showing top 20 rows



In [17]:
borough_year_sql = "SELECT B1,YEAR(CAST(TO_DATE(CRASH_DATE,'MM/dd/yyyy') as DATE)) YEAR, COUNT(*) as COUNT from allData"
borough_year_sql += " GROUP BY B1,YEAR ORDER BY B1,COUNT"

In [18]:
borough_year_df = spark.sql(borough_year_sql)
borough_year_df.show()

+--------+----+-----+
|      B1|YEAR|COUNT|
+--------+----+-----+
|   BRONX|2012|10104|
|   BRONX|2023|10862|
|   BRONX|2022|14334|
|   BRONX|2021|15525|
|   BRONX|2020|16142|
|   BRONX|2014|20816|
|   BRONX|2013|20827|
|   BRONX|2015|22498|
|   BRONX|2016|25606|
|   BRONX|2019|25614|
|   BRONX|2017|26960|
|   BRONX|2018|28687|
|BROOKLYN|2023|23370|
|BROOKLYN|2012|24448|
|BROOKLYN|2022|27867|
|BROOKLYN|2021|30483|
|BROOKLYN|2020|31278|
|BROOKLYN|2013|49241|
|BROOKLYN|2014|49969|
|BROOKLYN|2015|53368|
+--------+----+-----+
only showing top 20 rows



In [19]:
borough_month_sql = "SELECT B1,MONTH(CAST(TO_DATE(CRASH_DATE,'MM/dd/yyyy') as DATE)) MONTH, COUNT(*) as COUNT from allData"
borough_month_sql += " GROUP BY B1,MONTH ORDER BY B1,COUNT"

In [20]:
borough_month_df = spark.sql(borough_month_sql)
borough_month_df.show()

+--------+-----+-----+
|      B1|MONTH|COUNT|
+--------+-----+-----+
|   BRONX|    2|17197|
|   BRONX|    4|17499|
|   BRONX|    1|18708|
|   BRONX|    3|19313|
|   BRONX|   11|19881|
|   BRONX|   12|19907|
|   BRONX|    5|20238|
|   BRONX|    6|20248|
|   BRONX|    9|20893|
|   BRONX|    8|21099|
|   BRONX|   10|21284|
|   BRONX|    7|21708|
|BROOKLYN|    2|36832|
|BROOKLYN|    4|38607|
|BROOKLYN|    1|40428|
|BROOKLYN|    3|41982|
|BROOKLYN|   12|43675|
|BROOKLYN|   11|44394|
|BROOKLYN|    5|44515|
|BROOKLYN|    6|44907|
+--------+-----+-----+
only showing top 20 rows



In [21]:
person_data = spark.read.format('csv').options(header='true',inferschema='true').load("Person_data.csv")

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/C:/Users/321ni/Desktop/Year 2023- 2024/Big Data C/Person_data.csv.

In [ ]:
person_data.count()

In [ ]:
#Modified and added vehicle type details, need to rerun
merge_cols = ['COLLISION_ID','B1','NUMBER_OF_PERSONS_INJURED',
 'NUMBER_OF_PERSONS_KILLED',
 'NUMBER_OF_PEDESTRIANS_INJURED',
 'NUMBER_OF_PEDESTRIANS_KILLED',
 'NUMBER_OF_CYCLIST_INJURED',
 'NUMBER_OF_CYCLIST_KILLED',
 'NUMBER_OF_MOTORIST_INJURED',
 'NUMBER_OF_MOTORIST_KILLED',"VEHICLE_TYPE_CODE_1","VEHICLE_TYPE_CODE_2","VEHICLE_TYPE_CODE_3","VEHICLE_TYPE_CODE_4"
             ,"VEHICLE_TYPE_CODE_5"]

In [ ]:
person_data = person_data.withColumnRenamed("COLLISION_ID","COLLISION_ID_PERSON")

In [ ]:
merged_data = person_data.join(mvcdata2[merge_cols],person_data.COLLISION_ID_PERSON == mvcdata2.COLLISION_ID,"inner")

In [ ]:
merged_data.count()

In [ ]:
merged_data.columns

In [ ]:
#merged_data.repartition(1).write.format('csv').options(header='true').save("person_merged_accident")

In [ ]:
merged_data.createOrReplaceTempView("merged_data")

In [ ]:
merged_data.count

In [ ]:
distinct_SAFETY_EQUIPMENT = "SELECT SAFETY_EQUIPMENT,COUNT(*) FROM merged_data GROUP BY SAFETY_EQUIPMENT"
distinct_PERSON_INJURY = "SELECT PERSON_INJURY,COUNT(*) FROM merged_data GROUP BY PERSON_INJURY"
distinct_PERSON_TYPE = "SELECT PERSON_TYPE,COUNT(*) FROM merged_data GROUP BY PERSON_TYPE"
distinct_EJECTION = "SELECT EJECTION,COUNT(*) FROM merged_data GROUP BY EJECTION"
distinct_POSITION_IN_VEHICLE = "SELECT POSITION_IN_VEHICLE,COUNT(*) FROM merged_data GROUP BY POSITION_IN_VEHICLE"
distinct_PERSON_SEX = "SELECT PERSON_SEX,COUNT(*) FROM merged_data GROUP BY PERSON_SEX"

In [ ]:
distinct_PED_LOCATION = "SELECT PED_LOCATION,COUNT(*) FROM merged_data GROUP BY PED_LOCATION"
distinct_PED_ACTION = "SELECT PED_ACTION,COUNT(*) FROM merged_data GROUP BY PED_ACTION"

In [ ]:
distinct_SAFETY_EQUIPMENT_df = spark.sql(distinct_SAFETY_EQUIPMENT)
distinct_SAFETY_EQUIPMENT_df.show()

In [ ]:
distinct_PERSON_INJURY_df = spark.sql(distinct_PERSON_INJURY)
distinct_PERSON_INJURY_df.show()


In [ ]:
distinct_PERSON_TYPE_df = spark.sql(distinct_PERSON_TYPE)
distinct_PERSON_TYPE_df.show()

In [ ]:
distinct_EJECTION_df = spark.sql(distinct_EJECTION)
distinct_EJECTION_df.show()

In [ ]:
distinct_POSITION_IN_VEHICLE_df = spark.sql(distinct_POSITION_IN_VEHICLE)
distinct_POSITION_IN_VEHICLE_df.show()

In [ ]:
distinct_PERSON_SEX_df = spark.sql(distinct_PERSON_SEX)
distinct_PERSON_SEX_df.show()

In [ ]:
distinct_PED_LOCATION_df = spark.sql(distinct_PED_LOCATION)
distinct_PED_LOCATION_df.show()

In [ ]:
distinct_PED_ACTION_df = spark.sql(distinct_PED_ACTION)
distinct_PED_ACTION_df.show()

In [ ]:
driver_gender_sql = "SELECT PERSON_SEX, COUNT(*) FROM merged_data where POSITION_IN_VEHICLE = 'Driver' GROUP BY PERSON_SEX"

In [ ]:
driver_gender_df = spark.sql(driver_gender_sql)
driver_gender_df.show()

In [ ]:
contributing_factor1_sql = "SELECT CONTRIBUTING_FACTOR_1,COUNT(*) as Total from merged_data "
contributing_factor1_sql += " WHERE CONTRIBUTING_FACTOR_1 IS NOT NULL GROUP BY CONTRIBUTING_FACTOR_1 ORDER BY Total DESC"
contributing_factor2_sql = "SELECT CONTRIBUTING_FACTOR_2,COUNT(*) as Total from merged_data "
contributing_factor2_sql += " WHERE CONTRIBUTING_FACTOR_2 IS NOT NULL GROUP BY CONTRIBUTING_FACTOR_2 ORDER BY Total DESC"

In [ ]:
contributing_factor1_df = spark.sql(contributing_factor1_sql)
contributing_factor1_df.show()

In [ ]:
contributing_factor2_df = spark.sql(contributing_factor2_sql)
contributing_factor2_df.show()

In [ ]:
contributing_factor1_df.count()

In [ ]:
contributing_factor2_df.count()

In [ ]:
#Find the number of vehicles involved in collisions
num_vehicle_sql = "SELECT COLLISION_ID, COUNT(*) as total FROM merged_data WHERE POSITION_IN_VEHICLE = 'Driver' GROUP BY COLLISION_ID ORDER BY TOTAL DESC"

In [ ]:
num_vehicle_df = spark.sql(num_vehicle_sql)
num_vehicle_df.show()

In [ ]:
data_verify_sql = "SELECT * FROM allData WHERE COLLISION_ID = 4366219"
data_verify_df = spark.sql(data_verify_sql)
data_verify_df.show()